# Create an API

## Create a root endpoint

In [0]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def root():
    return {"greeting": "hello"}


## Add CORS

In [0]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

## Add a predict endpoint

In [0]:
@app.get("/predict")
def predict(pickup_datetime,  # 2013-07-06 17:18:00
            lon1,             # -73.950655
            lat1,             # 40.783282
            lon2,             # -73.984365
            lat2,             # 40.769802
            passcount):       # 1
    return {"pred": "the prediction"}


## Build a DataFrame from the API parameters

In [0]:
import pandas as pd

# ⚠️ no timezone conversion here: the user is assumed to provide an UTC datetime

X_pred = pd.DataFrame({
    "key": ["truc"],
    "pickup_datetime": [pickup_datetime + " UTC"],
    "pickup_longitude": [float(lon1)],
    "pickup_latitude": [float(lat1)],
    "dropoff_longitude": [float(lon2)],
    "dropoff_latitude": [float(lat2)],
    "passenger_count": [int(passcount)]})


## Load a trained model and make a prediction

In [0]:
import joblib

# load the trained model
pipeline = joblib.load("model.joblib")

# make a prediction for the parameters passed to the API
y_pred = pipeline.predict(X_pred)

# return the prediction
prediction = y_pred[0]


# Put API in production

## Create Dockerfile

In [0]:
FROM python:3.8-buster

COPY api api
COPY taxifare taxifare
COPY model.joblib model.joblib
COPY requirements.txt requirements.txt

RUN pip install --upgrade pip
RUN pip install -r requirements.txt

CMD uvicorn api.fast:app --host 0.0.0.0 --port $PORT
